In [1]:
import cv2
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.signal import convolve2d
from statistics import mode 
from math import sqrt

In [2]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [3]:
def opening(img,ST):
    imgEroded=cv2.erode(img,ST)
    return cv2.dilate(imgEroded,ST)
def closing(img,ST):
    imgDilated=cv2.dilate(img,ST)
    return cv2.erode(imgDilated,ST)

In [10]:
img= cv2.imread("imgcropped.jpg",0) 

# cv2.imwrite("test.jpg",img)
# print(img[:,1])
if(np.amax(img)==1):
    # img=img.astype("uint8")
    # cv2.imwrite("test1.jpg",img)
    print("hi")
else:
    img=(img/255)
    # cv2.imwrite("test2.png",img*255)
# print(img[:,1])
img2= np.copy(img)
img=1-img


# cv2.imwrite("test3.jpg",img*255)
# print(img)
proj = np.sum(img,1)
#proj=1-proj
maxProjection = np.max(proj) 
w = 500 
result = np.zeros(img.shape) 
result2 = np.zeros(img.shape) 
width=img.shape[1] 
binOrig=np.copy(img) 
# Draw a line for each row 
for row in range(img.shape[0]):
    result2[row,0:int(proj[row])]=1
    if(proj[row]>.8*maxProjection): 
        cv2.line(result, (0,row), (int(proj[row]*(maxProjection/width)),row), (255,255,255), 1) 
        
cv2.imwrite('result2.png', result2*255) 
cv2.imwrite('result.png', result) 
result=result2
result=1-result
print(proj.shape, img.shape)

(315,) (315, 945)


In [11]:
def calcStaffPos(rowHist):
    n,m=rowHist.shape
    thres=int((maxProjection)/2)
    print(thres,m)
    start=end=0
    print(rowHist)
    width=[]
    #rowHist=(rowHist/255).astype("uint8")
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                print(start,end)
                print((start+end)/2)
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    
    return np.array(width),np.array(peaksMid)

width,peaksMids=calcStaffPos(result)
print(width,peaksMids)

151 945
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
49 54
51.5
57 62
59.5
65 71
68.0
73 79
76.0
82 87
84.5
188 193
190.5
196 202
199.0
204 210
207.0
213 218
215.5
221 227
224.0
[5 5 6 6 5 5 6 6 5 6] [ 51  59  68  76  84 190 199 207 215 224]


In [12]:
def removeStaffRow(imgOriginal,midPoint,curWidth):
    thresPixel=curWidth
    for i in range(imgOriginal.shape[1]):
        pixelSum= sum(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
        if(pixelSum<=thresPixel):
            print(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
            imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1]=0
    return imgOriginal


# img= cv2.imread("note.png",0) 
# img=img/255
# img=img.astype('uint8')
# imgCopy=np.copy(img)
# cv2.imwrite("imgCopy.png",img*255)
img2=1-img2
for i in range(len(peaksMids)):
    img2=removeStaffRow(img2,peaksMids[i],width[i])
cv2.imwrite("outing.png",img2*255)
    

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]


 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.00784314]
 [0.        ]
 [0.65882353]
 [0.59215686]
 [0.        ]
 [0.05490196]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.00784314]
 [0.        ]
 [0.65490196]
 [0.59215686]
 [0.        ]
 [0.05882353]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01176471]
 [0.        ]
 [0.65098039]
 [0.58823529]
 [0.        ]
 [0.0627451 ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01176471]
 [0.        ]
 [0.64705882]
 [0.58823529]
 [0.        ]
 [0.06666667]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01568627]
 [0.        ]
 [0.64313725]
 [0.58823529]
 [0.        ]
 [0.06666667]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01568627]
 [0.        ]
 [0.63921569]
 [0.58823529]
 [0.        ]
 [0.07058824]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01568627]
 [0.        ]
 [0.63921569]
 [0.58823529]
 [0.        ]
 [0.07058824]
 [0.        ]

 [0.        ]]
[[0.        ]
 [0.        ]
 [0.01176471]
 [0.25490196]
 [0.89411765]
 [0.        ]
 [0.04705882]
 [0.        ]
 [0.01568627]
 [0.01176471]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.23529412]
 [0.84313725]
 [0.        ]
 [0.0627451 ]
 [0.01176471]
 [0.01176471]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.34117647]
 [0.89411765]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.28627451]
 [0.77254902]
 [0.07058824]
 [0.20392157]
 [0.16470588]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.2       ]
 [0.85490196]
 [0.56862745]
 [0.93333333]
 [0.74901961]
 [0.06666667]
 [0.01960784]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.35686275]
 [1.        ]
 [0.67843137]
 [1.        ]
 [0.87058824]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.54117647]
 [0.76078431]
 [1.        ]
 [0.2627451 ]
 [0.65098039]
 [0.83921569]
 [0.02745098]
 [0.        ]

 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01960784]
 [0.        ]
 [0.07058824]
 [0.        ]
 [0.8627451 ]
 [0.18823529]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01960784]
 [0.        ]
 [0.07058824]
 [0.        ]
 [0.8627451 ]
 [0.18823529]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01960784]
 [0.        ]
 [0.07058824]
 [0.        ]
 [0.8627451 ]
 [0.18823529]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01960784]
 [0.        ]
 [0.07058824]
 [0.        ]
 [0.8627451 ]
 [0.18823529]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01960784]
 [0.        ]
 [0.07058824]
 [0.        ]
 [0.8627451 ]
 [0.18823529]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01960784]
 [0.        ]
 [0.07058824]
 [0.        ]
 [0.8627451 ]
 [0.18823529]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01960784]
 [0.        ]
 [0.07058824]
 [0.        ]
 [0.8627451 ]
 [0.18823529]
 [0.        ]

 [0.07843137]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.08235294]
 [0.        ]
 [0.76078431]
 [0.32156863]
 [0.        ]
 [0.        ]
 [0.02745098]
 [0.        ]
 [0.07058824]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.08235294]
 [0.        ]
 [0.76470588]
 [0.32156863]
 [0.        ]
 [0.        ]
 [0.02745098]
 [0.        ]
 [0.07058824]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.08235294]
 [0.        ]
 [0.76470588]
 [0.3254902 ]
 [0.        ]
 [0.        ]
 [0.03137255]
 [0.        ]
 [0.0745098 ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.08235294]
 [0.        ]
 [0.76470588]
 [0.3254902 ]
 [0.        ]
 [0.        ]
 [0.03137255]
 [0.        ]
 [0.08627451]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.09019608]
 [0.        ]
 [0.77254902]
 [0.31372549]
 [0.        ]
 [0.        ]
 [0.01960784]
 [0.        ]
 [0.07843137]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.09019608]
 [0.        ]
 [0.77254902]
 [0.31764706]
 [0.        ]
 [0.        ]
 [0.01960784]


...

 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03921569]
 [0.        ]
 [0.92156863]
 [0.05098039]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03921569]
 [0.        ]
 [0.92156863]
 [0.05098039]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.04313725]
 [0.        ]
 [0.9254902 ]
 [0.05490196]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.04313725]
 [0.        ]
 [0.93333333]
 [0.0627451 ]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.04313725]
 [0.        ]
 [0.94117647]
 [0.07058824]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01568627]
 [0.03137255]
 [0.        ]
 [0.92941176]
 [0.10588235]
 [0.        ]
 [0.05098039]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.01176471]
 [0.05882353]
 [0.        ]
 [0.9372549 ]
 [0.11764706]
 [0.        ]
 [0.02745098]

 [0.        ]]
[[0.        ]
 [0.        ]
 [0.05882353]
 [0.        ]
 [0.88627451]
 [0.12156863]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.05882353]
 [0.        ]
 [0.88627451]
 [0.12156863]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.05882353]
 [0.        ]
 [0.88627451]
 [0.12156863]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.05882353]
 [0.        ]
 [0.88627451]
 [0.12156863]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.05882353]
 [0.        ]
 [0.88627451]
 [0.12156863]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.05882353]
 [0.        ]
 [0.88627451]
 [0.12156863]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]


 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.93333333]
 [0.35294118]
 [0.        ]
 [0.        ]
 [0.00784314]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.9254902 ]
 [0.36078431]
 [0.        ]
 [0.        ]
 [0.00784314]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.00392157]
 [0.91764706]
 [0.36470588]
 [0.        ]
 [0.        ]
 [0.01176471]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.00392157]
 [0.90980392]
 [0.37647059]
 [0.        ]
 [0.        ]
 [0.01568627]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.00392157]
 [0.90196078]
 [0.38431373]
 [0.        ]
 [0.        ]
 [0.01568627]
 [0.        ]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.89803922]
 [0.38823529]
 [0.        ]
 [0.        ]


 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


True